In [1]:
import pandas as pd
import numpy as np

import os
import dotenv
from langchain_core.prompts import PromptTemplate
from langchain import LLMChain
from langchain_openai import OpenAI
from langchain.callbacks import get_openai_callback
import json

template = """I'll pass you a cocktail name and its ingredients with a\
    brandname and you need to return in a json, the base liquor and its\
    ingredients without measures and for example:\
    cocktail:West Coast Punch
    ingredients: 1.5 oz Junipero Gin, .75 oz Pineapple Juice,\
    .5 oz Monin Strawberry Puree, 6-7  Mint Leaves, .5 oz Lemon Sour, 1 oz Lemon Lime Soda
    return:
    
   {{"base_liquor":"gin","ingredients":"Junipero Gin, Pineapple Juice,\
    Monin Strawberry Puree, Mint Leaves, Lemon Sour, Lemon Lime Soda"}}
    
    
    cocktail name: {cocktail}\
    list of ingredients: {ingredients}
"""

print(os.getcwd())

/Users/rafaeldavila/Documents/Duke/Sem2/Cloud/final_project/MIDS_MIX/00_code


In [19]:
os.listdir("../../data")

['hotaling_cocktails_Cocktails.csv']

In [21]:
data.shape

(687, 9)

In [24]:
data = pd.read_csv("../../data/hotaling_cocktails_Cocktails.csv")
responses = []
counter = 0

for _, row in data.iterrows():

    prompt = PromptTemplate(
        input_variables=["cocktail", "ingredients"], template=template
    )
    llm_gpt3_5_chain = LLMChain(prompt=prompt, llm=OpenAI())

    input_params = {
        "cocktail": row["Cocktail Name"],
        "ingredients": row["Ingredients"],
        "temperature": 0,
    }
    response = llm_gpt3_5_chain.invoke(input_params)
    responses.append(response["text"])

    counter += 1
    if counter % 50 == 0:
        print(f"Number: {counter}")

Number: 50
Number: 100
Number: 150
Number: 200
Number: 250
Number: 300
Number: 350
Number: 400
Number: 450
Number: 500
Number: 550
Number: 600
Number: 650


In [27]:
import pandas as pd
import re


def get_base_liquor(responses):
    base_liquor = []
    for response in responses:
        match = re.search(r'"base_liquor":"(.*?)"', response)
        if match:
            base_liquor.append(match.group(1))
        else:
            base_liquor.append(None)
    return base_liquor


def get_ingredients(responses):
    ingredients = []
    for response in responses:
        match = re.search(r'"ingredients":"(.*?)"', response)
        if match:
            ingredients.append(match.group(1))
        else:
            ingredients.append(None)
    return ingredients


data["base_liquor"] = get_base_liquor(responses)
data["cleaned_ingredients"] = get_ingredients(responses)

In [28]:
data.sample(10)

,Cocktail Name,Bartender,Bar/Company,Location,Ingredients,Garnish,Glassware,Preparation,Notes,base_liquor,cleaned_ingredients
10,Happiness,Danny Louie,Mister Jiu's,San Francisco,"1.5 oz Jasmine Tea/Rice-Infused Junipero Gin*,...",Edible Flower,Coupe,"Add all ingredients in a cocktail shaker, shak...",Junipero Gin 20th Anniversary Signature Cocktail,gin,"Jasmine Tea, Rice, Sour Apple Liqueur, Lime, H..."
456,"Eat, Drink and Be Cherry",Paul Taylor,Cherry Blossom PUB,Washington DC,".75 oz Nikka Coffey Grain Whisky, .75 oz Luxar...",3 Cherries Picked,Bamboo Highball Glass,"Shake everything except for the soda water, st...",NaN,whisky,"Nikka Coffey Grain Whisky, Luxardo Sangue Morl..."
274,Prickly Pear Margarita,Edgar & Joselino Solis,Matador,San Francisco,"2 oz Chinaco Blanco Tequila, 1 oz Lime Juice, ...",Mint Sprig,Margarita Glass,Mint salt on the rim. Garnish with mint sprig.,NaN,tequila,"Chinaco Blanco Tequila, Lime Juice, Prickly Pe..."
517,Pisco Punch (Adaptation by The Comstock Saloon),NaN,Comstock Saloon,San Francisco,"1.5 oz BarSol Selecto Italia Pisco, .75 oz Sma...",Lemon Peel,Flip Glass,"Shake, strain into a chilled flip glass with o...",Featured on Speed Rack 2015 SF Speed City Take...,pisco,"BarSol Selecto Italia Pisco, Small Hand Foods ..."
314,Sage & Sound,Brian Means,Michael Mina Restaurant Group,San Francisco,"1.5 oz Junipero Gin, 1 oz White Port, .75 oz L...",Whole Star Anise,Coupe,Muddle the sage in a mixing tin. Add the remai...,Junipero Gin 20th Anniversary Signature Cocktail,gin,"Junipero Gin, White Port, Lemon Juice, Honey C..."
24,The King's Autumn Cobbler,NaN,NaN,NaN,"1 oz King's Ginger Liqueur, .25 oz Briottet Cr...","3 Lemon Wedges, 3 Blackberries",NaN,Add all ingredients over ice. Stir with a bars...,NaN,ginger liqueur,"King's Ginger Liqueur, Briottet Creme de Mure,..."
624,Nigamono,John deBarry,Momofuku,NaN,"2 oz Nikka Coffey Grain Whisky, .75 oz Maurin ...",Luxardo Cherries,Double Old Fashioned Glass,Stir. Use large ice cube.,NaN,whisky,"Nikka Coffey Grain Whisky, Maurin Quina, Luxar..."
305,Sabotage,William Golde,NaN,NaN,"2 oz Breck Bourbon, .5 oz Dancing Pines Black ...","Burnt orange, Thyme sprig",Rocks Glass,"Muddle sage and Thyme. Stirred, over ice in si...",NaN,bourbon,"Breck Bourbon, Dancing Pines Black Walnut Bour..."
652,Speyside Highball,NaN,NaN,NaN,"1.5 oz Glenrothes Vintage Reserve, 1 oz Tuaca,...",Cucumber,Highball Glass,NaN,NaN,scotch,"Glenrothes Vintage Reserve, Tuaca, Lemon Juice..."
177,Elgin Elixir,Alan Kropf,NaN,San Francisco,"1 oz BenRiach Single Malt 12 YO, 1 oz C. Droui...",NaN,Rocks Glass,Combine ingredients with ice in a mixing glass...,NaN,whiskey,"BenRiach Single Malt 12 YO, C. Drouin Calva..."


In [38]:
data["base_liquor"] = [
    i.lower() if isinstance(i, str) else i for i in data["base_liquor"]
]
frequency_table = data["base_liquor"].value_counts()

In [42]:
data.to_csv("../../data/final_data.csv", index=False)

In [45]:
top_20_base_liquor = frequency_table[:20].index
filtered_data = data[data["base_liquor"].isin(top_20_base_liquor)]
# filtered_data.to_csv("../../data/filtered_data.csv", index=False)

In [46]:
filtered_data.shape

(604, 11)

# Adding Embeddings

In [22]:
from sentence_transformers import SentenceTransformer
import pandas as pd

data = pd.read_csv("../10_data/filtered_data.csv")

In [23]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Sentences are encoded by calling model.encode()
embeddings = [model.encode(v) for v in data["cleaned_ingredients"]]

In [24]:
embeddings[0]

array([-5.06987162e-02,  2.69325003e-02,  9.75127448e-04,  2.15670019e-02,
        4.10591327e-02,  2.37977859e-02,  1.25843108e-01,  1.11135114e-02,
       -5.69760203e-02, -2.07591522e-02, -3.30801718e-02, -6.24259673e-02,
       -2.43772753e-03, -3.96118462e-02, -1.45647256e-02, -3.04627139e-03,
        6.34545684e-02,  2.36449260e-02, -4.14629467e-02, -4.96409908e-02,
        4.88925017e-02, -7.25222826e-02,  1.23532070e-02,  5.63482568e-02,
       -3.19382213e-02,  8.43499750e-02,  1.95802711e-02,  6.74890354e-02,
        1.13381501e-02, -1.21968083e-01,  2.43950691e-02,  1.10667534e-02,
        9.16486830e-02, -7.57726207e-02, -4.91396291e-04,  8.48208927e-03,
        4.04812545e-02, -8.62105042e-02, -4.95121591e-02,  1.31872604e-02,
        5.28577156e-02,  8.24941397e-02,  4.41931300e-02,  8.83641466e-02,
       -5.63975237e-02, -3.22612487e-02, -1.90778263e-02,  1.46013880e-02,
       -9.08279698e-03,  3.03754080e-02, -7.99879655e-02, -2.32875496e-02,
       -3.75914276e-02, -

In [25]:
data["embeddings"] = embeddings

In [11]:
# data.to_csv("../10_data/filtered_data.csv", index=False)

In [9]:
from pinecone import Pinecone
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("midsmix")

In [26]:
data.sample()

,ID,name,base_liquor,cleaned_ingredients,embeddings
192,ID193,A Southern Jackass,whiskey,"Prichard's Lincoln County Lightning, King's Gi...","[-0.004616389, -0.059094407, -0.001914403, 0.0..."


In [27]:
import ast

vectors = []
for _, row in data.iterrows():
    vectors.append(
        {
            "id": row["ID"],
            "values": row["embeddings"],
            "metadata": {"name": row["name"], "base": row["base_liquor"]},
        }
    )

In [29]:
index.upsert(vectors)

{'upserted_count': 603}

In [31]:
index.fetch(ids=["ID1"])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'ID1': {'id': 'ID1',
                     'metadata': {'base': 'mezcal', 'name': 'Flor de Amaras'},
                     'values': [-0.0506987162,
                                0.0269325,
                                0.000975127448,
                                0.0215670019,
                                0.0410591327,
                                0.0237977859,
                                0.125843108,
                                0.0111135114,
                                -0.0569760203,
                                -0.0207591522,
                                -0.0330801718,
                                -0.0624259673,
                                -0.00243772753,
                                -0.0396118462,
                                -0.0145647256,
                                -0.00304627139,
                                0.0634545684,
                                0.023644926,
    